In [13]:
import pandas as pd
import numpy as np
import string

In [2]:
df = pd.read_json("transaction-data-adhoc-analysis.json")
df

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date
0,USNS Wagner\nFPO AE 00917,1926/03/11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022/01/12
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022/01/12
2,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",5500,2022/01/08
3,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Yummy Vegetables,(x1)",500,2022/01/10
4,"33911 Hansen Loaf\nWest Sandra, ND 54188",1949/09/27,michael78@yahoo.com,Kristen Cooper,F,qhernandez,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2697,2022/01/24
...,...,...,...,...,...,...,...,...,...
83031,"96081 Harrison Brooks\nJenniferbury, MA 02124",2019/10/26,christina55@hotmail.com,Tammy Byrd,F,kmcconnell,"Exotic Extras,Kimchi and Seaweed,(x2);HealthyK...",3588,2022/06/05
83032,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"HealthyKid 3+,Yummy Vegetables,(x1);Exotic Ext...",2095,2022/06/12
83033,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"Candy City,Orange Beans,(x4);Candy City,Gummy ...",3396,2022/06/15
83034,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"HealthyKid 3+,Nutrional Milk,(x2);Candy City,G...",4280,2022/06/09


In [3]:
# fixing the "grain"
# new row everytime there's a new item ordered
df = df.assign(transaction_items=df['transaction_items'].str.split(';')).explode('transaction_items')

# split that transaction_items further to columns, like item_brand, item_type, items_count
df[['item_brand','item_type','item_count']] = df['transaction_items'].str.split(',', n=2, expand=True)

# move columns
df = df[['address', 'birthdate', 'mail', 'name', 'sex', 'username', 'transaction_items', 'item_brand', 'item_type', 'item_count', 'transaction_value', 'transaction_date']]

# only have numbers in item_count
df['item_count'] = df['item_count'].str.replace('(x','',regex=False).str.replace(')','',regex=False).apply(pd.to_numeric)
df

,address,birthdate,mail,name,sex,username,transaction_items,item_brand,item_type,item_count,transaction_value,transaction_date
0,USNS Wagner\nFPO AE 00917,1926/03/11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",Exotic Extras,Beef Chicharon,4,5196,2022/01/12
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3)",Exotic Extras,Beef Chicharon,3,12056,2022/01/12
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Nutrional Milk,(x4)",HealthyKid 3+,Nutrional Milk,4,12056,2022/01/12
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Candy City,Orange Beans,(x1)",Candy City,Orange Beans,1,12056,2022/01/12
2,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Gummy Vitamins,(x3)",HealthyKid 3+,Gummy Vitamins,3,5500,2022/01/08
...,...,...,...,...,...,...,...,...,...,...,...,...
83034,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"HealthyKid 3+,Nutrional Milk,(x2)",HealthyKid 3+,Nutrional Milk,2,4280,2022/06/09
83034,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"Candy City,Gummy Worms,(x2)",Candy City,Gummy Worms,2,4280,2022/06/09
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012/04/30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1)",HealthyKid 3+,Nutrional Milk,1,2639,2022/06/03
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012/04/30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"Candy City,Gummy Worms,(x3)",Candy City,Gummy Worms,3,2639,2022/06/03


In [4]:
# count total number of items total
df["item_count"].sum()

415150

In [5]:
# convert transaction_date from object to datetime?
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# group by month
total_item_count = df.groupby(df['transaction_date'].dt.month).sum()

# transpose to make rows the columns
total_item_count.T

transaction_date,1,2,3,4,5,6
item_count,68041,69910,69297,69257,69130,69515
transaction_value,144894308,148714957,149274236,149057455,148941801,148938336


In [6]:
# fixing the grain added duplicate rows, so let's solve that

# convert transaction_date from object to datetime
df['month'] = df['transaction_date'].dt.month

# use max function to remove dupes (mean would return float)
total_transaction_value = df.groupby(df.index).max('transaction_date')

# use sum function to get the total transaction value
pivot_table = total_transaction_value.groupby('month').sum()

pivot_table

,item_count,transaction_value
month,,
1,41394,62522565
2,42155,63820403
3,41844,63943846
4,41675,63420965
5,41801,63740675
6,41987,63704318


In [7]:
# repeaters
repeaters = []
for i in range(1,7): # months 1 to 6
    customers_before = set(df.loc[df['month']==i-1]['username']) # get all unique customers who bought in month i-1
    customers_current = set(df.loc[df['month']==i]['username']) # get all unique customers who bought in month i
    repeaters.append(len(customers_before.intersection(customers_current))) # then do intersection, which will leave us with the customers who bought bought
    
pivot_table["repeaters"] = repeaters
pivot_table

,item_count,transaction_value,repeaters
month,,,
1,41394,62522565,0
2,42155,63820403,5167
3,41844,63943846,5239
4,41675,63420965,5191
5,41801,63740675,5126
6,41987,63704318,5208


In [14]:
# inactive
inactive = []
for i in range(1,7):
    customers_before = set(df.loc[df['month']<i]['username'])
    customers_current = set(df.loc[df['month']==i]['username'])
    inactive.append(len(customers_before.difference(customers_current)))

pivot_table["inactive"] = inactive
pivot_table

,item_count,transaction_value,repeaters,inactive
month,,,,
1,41394,62522565,0,0
2,42155,63820403,5167,1416
3,41844,63943846,5239,1728
4,41675,63420965,5191,1912
5,41801,63740675,5126,1915
6,41987,63704318,5208,1840


In [16]:
# engaged
users_months = df.groupby('username').agg(total_distinct_months=('month','nunique'))

# now leave only users which have all months, so, 6
engaged_users = users_months.loc[users_months['total_distinct_months'] == 6]

# now count number of such users, it will be our answer
len(engaged_users)

2210

In [17]:
# place it on the pivot table
engaged = ['0','0','0','0','0',len(engaged_users)]

pivot_table["engaged"] = engaged
pivot_table

,item_count,transaction_value,repeaters,inactive,engaged
month,,,,,
1,41394,62522565,0,0,0
2,42155,63820403,5167,1416,0
3,41844,63943846,5239,1728,0
4,41675,63420965,5191,1912,0
5,41801,63740675,5126,1915,0
6,41987,63704318,5208,1840,2210


In [ ]:
df.groupby(['item_type', 'transaction_value', 'transaction_date']).max()

In [ ]:
# other metrics

# item_count of each item_type
df.groupby(["item_type","item_count"]).count()
df["orders"] = df["address"].count()
df[['item_type','item_count','orders']].groupby(["item_type","item_count"]).count()

In [ ]:
pivot_table["transaction_value"]

In [ ]:
# data visualization
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn import datasets, linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import numpy as np

# Now to create some histograms
# The plot on the left illustrates the distribution of songs based on their energy levels
# whereas the one on the right is a heatmap that illustrates the number of songs found at all values of valence and danceability.

x = "repeaters"
y = "transaction_value"

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=False, sharex=False, figsize=(10, 5))
fig.suptitle("Histograms")
h = ax2.hist2d(recommendations[x], recommendations[y], bins=20)
ax1.hist(recommendations["energy"])

ax2.set_xlabel(x)
ax2.set_ylabel(y)

ax1.set_xlabel("energy")

plt.colorbar(h[3], ax=ax2)

plt.show()

In [75]:

# matplotlib visuals
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline